# CSCI 5922 (Spring 2020) Final Project
name: Timothy Mason
***
Music Genre classification using a CRNN as described in the paper blah, blah, blah...

todo: fill in citations and flesh out overview

In [ ]:
import os
import numpy as np
import pandas as pd
import hdf5_getters
from tqdm.notebook import tqdm      # for easy and attractive progress bars

def get_file_list( msd_path='./MillionSongSubset/data' ):
    """
    Walk the directory structure of the provided path, and return a list of the paths to
    all *.h5 files

    Parameters
    ----------
    msd_path: The path to the root directory of the MSD dataset

    Returns
    -------
    a list of path strings to each file in the MSD data set
    """
    file_list = []

    for root, dirs, files in os.walk(msd_path):
        for f in files:
            if f[-3:] == ".h5":
                file_list.append(os.path.join(root,f))

    return file_list

def get_data(
              # categories=(
              #     "num_songs",
              #       "artist_familiarity",
              #       "artist_hotttnesss",
              #       "artist_id",
              #       "artist_mbid",
              #       "artist_playmeid",
              #       "artist_7digitalid",
              #       "artist_latitude",
              #       "artist_longitude",
              #       "artist_location",
              #       "artist_name",
              #       "release",
              #       "release_7digitalid",
              #       "song_id",
              #       "song_hotttnesss",
              #       "title",
              #       "track_7digitalid",
              #       "similar_artists",
              #       "artist_terms",
              #       "artist_terms_freq",
              #       "artist_terms_weight",
              #       "analysis_sample_rate",
              #       "audio_md5",
              #       "danceability",
              #       "duration",
              #       "end_of_fade_in",
              #       "energy",
              #       "key",
              #       "key_confidence",
              #       "loudness",
              #       "mode",
              #       "mode_confidence",
              #       "start_of_fade_out",
              #       "tempo",
              #       "time_signature",
              #       "time_signature_confidence",
              #       "track_id",
              #       "segments_start",
              #       "segments_confidence",
              #       "segments_pitches",
              #       "segments_timbre",
              #       "segments_loudness_max",
              #       "segments_loudness_max_time",
              #       "segments_loudness_start",
              #       "sections_start",
              #       "sections_confidence",
              #       "beats_start",
              #       "beats_confidence",
              #       "bars_start",
              #       "bars_confidence",
              #       "tatums_start",
              #       "tatums_confidence",
              #       "artist_mbtags",
              #       "artist_mbtags_count",
              #       "year"
              # ),
              categories=("artist_name", "release", "segments_pitches",
                           "segments_timbre", "tatums_start", "title", "year"),
              msd_path='./MillionSongSubset/data',
              first = 0, last = None):
    """
    Walk the directory structure of the provided path, and read data from the
    *.h5 files in all subdirectories.

    Parameters
    ----------
    categories: list of Strings
        a list of keyword strings
    msd_path: The path to the root directory of the MSD dataset
    first: Index of the first file to read
    last: Index of the last file to read.  If None, then read entire list of files

    Returns
    -------
    a Pandas dataframe object containing the specified categories of MSD data
    """

    print(f"Reading categories {categories} from {msd_path}/...")
    file_list = get_file_list(msd_path)

    if last == None:
        last = len(file_list)
    data = []

    for f in tqdm(file_list[first:last], desc="Reading files {first} through {last}"):
        h5file = hdf5_getters.open_h5_file_read(f)
        d = {}
        for c in categories:
            t = getattr(hdf5_getters, "get_"+c)(h5file)
            d[c] = t
        h5file.close()
        data.append(d)
    df = pd.DataFrame(data)
    return df

df = get_data(last=10000)
print(df)